# Physical constants and units
### phys481_week08a_constants_and_units

In [8]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy

## Scipy constants
https://docs.scipy.org/doc/scipy/reference/constants.html

This package defines variables containing many useful physical and mathematical constants.  
 
It is generally a bad idea to import the entire contents of a package, as new values may "step on" (overwrite) existing ones

    from scipy.constants import *
    
Even importing a single value is potentially troublesome.

In [9]:
h = 1.0
print('a variable named "h": ', h)

from scipy.constants import h
print('a variable named "h": ', h)
print('Original Plank', h)

h = 3.1415
print('Overwrite Plank: ', h)

from scipy.constants import h
print('Re-load Plank: ', h)

a variable named "h":  1.0
a variable named "h":  6.62607015e-34
Original Plank 6.62607015e-34
Overwrite Plank:  3.1415
Re-load Plank:  6.62607015e-34


There may appear to be two different values for the speed of light in a vaccuum

In [10]:
import scipy.constants
print('c: ', scipy.constants.c)
print('speed_of_light', scipy.constants.speed_of_light)

c:  299792458.0
speed_of_light 299792458.0


but a closer look shows that the values are not only equal, but actually identical ie. share the same value

In [11]:
print('values are equal: ', scipy.constants.c == scipy.constants.speed_of_light)
print('values are identical: ', scipy.constants.c is scipy.constants.speed_of_light)

values are equal:  True
values are identical:  True


#### getattr: get attributes programatically

In [12]:
name = 'speed_of_light'
getattr(scipy.constants, name, None)

299792458.0

In [14]:
value = getattr(scipy.constants,'mass_of_light', None)
if value is None:
    print('light has no mass')

light has no mass


### scipy.constants.physical_constants
https://docs.scipy.org/doc/scipy/reference/constants.html#Constants_database
Dictionary of physical constants, of the format 

    physical_constants[name] = (value, unit, uncertainty).

In [15]:
G = scipy.constants.physical_constants['Newtonian constant of gravitation']
print( G )

(6.6743e-11, 'm^3 kg^-1 s^-2', 1.5e-15)


## https://socialcompare.com/en/comparison/python-units-quantities-packages
"There are far too many different packages for unit conversion / quantities calculations in Python. Developers should cooperate on one package instead of fragmenting development across many different ones. This documents which features are supported by each package."

### Units with PINT
https://buildmedia.readthedocs.org/media/pdf/pint/0.7.2/pint.pdf

If we use `conda` to install a package called `PINT`

    conda install -c conda-forge pint
    
then we can attach units to values and undertake dimensional analysis.    

In [35]:
#% conda install -c conda-forge pint
#https://docs.scipy.org/doc/scipy/reference/constants.html
import scipy.constants
print( scipy.constants.G )
import pint

ureg = pint.UnitRegistry()
G = scipy.constants.physical_constants['Newtonian constant of gravitation']

def gravity_spherical(distance, mass):    
    earth_mass = 5.972e24 *ureg.kg
    return G[0] * ureg(G[1]) * earth_mass * mass / distance**2
force = gravity_spherical( 6371.2 * ureg.km, 1 * ureg.kg )
print('Force: ', force)

6.6743e-11
Force:  9819356.91321015 kilogram * meter ** 3 / kilometer ** 2 / second ** 2


In [37]:
# force in newtons
print( force, '\n', force.to('newton') )

9819356.91321015 kilogram * meter ** 3 / kilometer ** 2 / second ** 2 
 9.81935691321015 newton


In [36]:
# force does not have units of Joules
print( force, '\n', force.to('joule') )

DimensionalityError: Cannot convert from 'kilogram * meter ** 3 / kilometer ** 2 / second ** 2' ([length] * [mass] / [time] ** 2) to 'joule' ([length] ** 2 * [mass] / [time] ** 2)

#### Physical-type correctness in scientific Python
https://arxiv.org/ftp/arxiv/papers/1807/1807.07643.pdf
    
"We demonstrate the limitations of three Python unit-libraries and present
a justification and method for checking kind-of-quantity."




#### Unum
https://github.com/trzemecki/Unum

For a simple example, let's can calculate Usain Bolt's average speed during his record-breaking performance in the 2008 Summer Olympics

    >>> from unum.units import * # Load a number of common units.
    >>> distance = 100*m
    >>> time = 9.683*s
    >>> speed = distance / time
    >>> speed
    10.3273778788 [m/s]
    >>> speed.asUnit(mile/h)
    23.1017437978 [mile/h]

If we do something dimensionally incorrect, we get an exception rather than silently computing a correct result. Let's try calculating his kinetic energy using an erroneous formula

    >>> KE = 86*kg * speed / 2 # Should be speed squared!
    >>> KE.cast_unit(J)
    Traceback (most recent call last):
      File "<stdin>", line 1, in <module>
      File "unum\__init__.py", line 171, in asUnit
        s, o = self.matchUnits(other)
      File "unum\__init__.py", line 258, in matchUnits
        raise IncompatibleUnitsError(self, other)
    unum.IncompatibleUnitsError: [kg.m/s] can't be used with [J]